In [ ]:
from bs4 import BeautifulSoup as bs
import lxml
import os

In [ ]:
path = "dcp-data/letters/"
files = os.listdir(path)
print(files[1])
with open(path+files[1],"r") as file:
    content = file.readlines()
    print(content[0])
    content = "".join(content)
    bs_content = bs(content, "lxml")
help(bs_content)